In [3]:
import cv2
import numpy as np

In [10]:
#1. Use the following inputs to write your own​ ​2D convolution function. Verify your
    #implementation with ​OpenCV ​cv2.filter2D()​ ​function
    
#convolution function
# (-) values are replaced with 0 in the output image
def con2D(img, kernel, padding=1):
    #flipping the kernal because we getting the convolution not correlation
    kernel = np.flipud(np.fliplr(kernel))

    # extracting the dimensions from input and kernal
    Kernal_rows = kernel.shape[0]
    Kernal_cols = kernel.shape[1]
    Img_rows = img.shape[0]
    Img_cols = img.shape[1]

    # calculating the shape of convolved output
    output_rows = ((Img_rows - Kernal_rows + 2 * padding) ) + 1
    output_cols = ((Img_cols - Kernal_cols + 2 * padding) ) + 1
    output = np.zeros((output_rows, output_cols))

  
    # creating matrix with zeros with padding 
    padded_img = np.zeros((Img_rows + padding*2, Img_cols + padding*2)) 
    
    #updating the matrix with the given image
    padded_img[int(padding):int(-1 * padding), int(padding):int(-1 * padding)] = img  
   

    # Iterate through image
    for y in range(Img_cols):            
        for x in range(Img_rows):
            
            #calculating the value of kernal   
            k = (kernel *  padded_img[x: x +  Kernal_rows, y: y +Kernal_cols]).sum()    
            
            #if the k value is minus value, the value is replaced with zero(images doesn't have (-) pixel values)
            if k > 0:
                output[x,y] = k
            else:
                output[x,y] = 0                       

    return output

In [4]:
Y=np.array([[-1,-1,-1], [0,0,0], [1,1,1]])
Y

array([[-1, -1, -1],
       [ 0,  0,  0],
       [ 1,  1,  1]])

In [5]:
X = np.ones((8,8))*10
X

array([[10., 10., 10., 10., 10., 10., 10., 10.],
       [10., 10., 10., 10., 10., 10., 10., 10.],
       [10., 10., 10., 10., 10., 10., 10., 10.],
       [10., 10., 10., 10., 10., 10., 10., 10.],
       [10., 10., 10., 10., 10., 10., 10., 10.],
       [10., 10., 10., 10., 10., 10., 10., 10.],
       [10., 10., 10., 10., 10., 10., 10., 10.],
       [10., 10., 10., 10., 10., 10., 10., 10.]])

In [6]:
y0 = np.flipud(np.fliplr(Y))
print(cv2.filter2D(X,-1,y0,borderType=cv2.BORDER_CONSTANT))


[[-20. -30. -30. -30. -30. -30. -30. -20.]
 [  0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.]
 [ 20.  30.  30.  30.  30.  30.  30.  20.]]


In [14]:
print(con2D(X, Y, padding=1))

[[ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [20. 30. 30. 30. 30. 30. 30. 20.]]


In [15]:
img = cv2.imread('spunifnoisy.jpg', 0)
bilateral_IMG = cv2.bilateralFilter(img, 5, 200, 200, borderType=cv2.BORDER_CONSTANT)

cv2.imshow('bilateral', bilateral_IMG)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [16]:
#2. Bilateral,​ ​Gaussian and Median Filtering
#Bilateral filtering funtion

#this function claculates gaussian values for x value according to the sigma value
def gauss(x,sig):
    return (1.0/(2*np.pi*(sig**2)))*np.exp(-(x**2)/(2*(sig**2)))

#bilateral filtering funtion
def bilateral_fil(img,sigma_i, sigma_s):
    
    #getting the dimentions of the image
    img_rows = img.shape[0]
    img_cols = img.shape[1]
    output_image = np.zeros([img_rows, img_cols], dtype= 'uint8')
    
    mask_diameter = 5 # mask 5 x 5 
    
    for row in range(img_rows):
        for col in range(img_cols):
            
            #normalization constant
            C = 0
            #this will hold the value to add to the output image for rounding
            fil_image = 0
            
            for x in range(mask_diameter):
                for y in range(mask_diameter):
                    #getting index values according to the diameter of the mask
                    mx =row - (mask_diameter/2 - x)
                    my =col - (mask_diameter/2 - y)
                    
                    #checking whether the boundaries are exceeding
                    if mx >= img_rows:
                        mx = mx - img_rows
                    if my >= img_cols:
                        my = my - img_cols
                        
                    #calculating the guassian value for intensity
                    g_i= gauss(img[int(mx)][int(my)] - img[row][col], sigma_i)
                    
                    #calculating the gaussian value for spatial distance of pixels
                    g_s= gauss(np.sqrt(np.abs((mx-row)**2-(my-col)**2)), sigma_s)
                    
                    #getting weight according to the mask
                    wp = g_i * g_s
                    
                    #calculating total pixel value to add to the new image for position row,col
                    fil_image = (fil_image) + (img[int(mx)][int(my)] * wp)
                    
                    #calculating total value of weights to get the normalization constant
                    C = C + wp
                    
            #calculating the weighted average of nearby pixels
            fil_image = fil_image // C
            
            #adding the new pixel value after the filtering
            output_image[row][col] = int(np.round(fil_image))
            
    return output_image



In [17]:
image = cv2.imread("spunifnoisy.jpg",0)
new_img = cv2.GaussianBlur(image, (5,5),0, borderType=cv2.BORDER_CONSTANT)
cv2.imshow('openCV_Gaussian_filter', new_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [18]:
#A Gaussian filter with mask size 5 × 5 appropriate values of σ .
def Gaussian_fil(img, sigma): 
    fil_size = 5 # 5x5 mask is used
    
    #creating the np array to store calculated gaussian kernal
    gaussian_fil_kernal = np.zeros([fil_size, fil_size], np.float32)
    
    #getting the middle floor value according to the size of gaussian kernal
    i = fil_size//2
    j = fil_size//2
    
    for x in range(-i, i+1):
        for y in range(-j, j+1):
            
            #calculating the gaussian function 
            k1 = 2*np.pi*(sigma**2)
            k2 = np.exp(-(x**2 + y**2)/(2* sigma**2))
            gaussian_fil_kernal[x+i, y+j] = (1/k1)*k2
            
    #since we have to get convolution with original image, kernal is flipped horizontally and vertically
    #because cv2.filter2D() function give correlation, so we have to flip the kernal before give to that funtion
    y0 = np.flipud(np.fliplr(gaussian_fil_kernal))
    
    #getting the filtered image by getting convolution
    return  cv2.filter2D(img,-1,y0,borderType=cv2.BORDER_CONSTANT)               

In [19]:
img = cv2.imread("spunifnoisy.jpg",0)
new_img_gauss = Gaussian_fil(img, 1)
cv2.imshow('gaussian filtered image from own funtion', new_img_gauss)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [20]:
#A median filter of appropriate window size
def Median_fil(img, filter_size):
    
    #getting the sizes of image
    img_rows = img.shape[0]
    img_cols = img.shape[1]
    
    #dictionary to store the list of pixel values to get the median
    list_median = []
    
    
    output_image = np.zeros([img_rows, img_cols], dtype= 'uint8')
    index = filter_size // 2
    
    for i in range(img_rows):
        for j in range(img_cols):
            for k in range(filter_size):
                
                #checking the mask is in the corner and if it is in corner then 0 are appended
                if (i + k - index > img_rows - 1) or (i + k - index < 0):
                    for m in range(filter_size):
                        list_median.append(0)
                else:
                    #if the currnt cell of mask is outside the image then 0 is appended
                    if (j + index > img_cols - 1) or (j + k - index < 0):
                        list_median.append(0)
                    else:
                        #the pixel values of the mask which are inside the image are appended to the dictionary
                        for l in range(filter_size):
                            list_median.append(img[i + k - index][j + l - index])
            
            #sorting the extracted pixel from the mask to get the median value 
            list_median.sort()
            
            #setting the median value to the output image
            output_image[i][j] = list_median[len(list_median) // 2]
            
            #making empty the image to get the next filtered pixel value
            list_median = []
    return output_image



In [21]:
img = cv2.imread("spunifnoisy.jpg",0)
new_img_gauss = Median_fil(img, 3)
cv2.imshow('gaussian filtered image from own funtion', new_img_gauss)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [22]:
def main():
    img = cv2.imread("spunifnoisy.jpg",0)
    
    #1. Use the following inputs to write your own​ ​2D convolution function. Verify your
    #implementation with ​OpenCV ​cv2.filter2D()​ ​function
    
    X = np.ones((8,8))*10
    Y=np.array([[-1,-1,-1], [0,0,0], [1,1,1]])
    
    #my own funtion
    # minus values are removed from the output(Because images don't have - value pixels)
    print(con2D(X, Y, padding=1))
    print('\n\n')
    
    #inbuild cv2 funtion
    #in this cv2.filter2D gives correlation, not convolution. Therefore kernal is flipped horizontally and vertically
    #before adding to the funtion
    y0 = np.flipud(np.fliplr(Y))
    print(cv2.filter2D(X,-1,y0,borderType=cv2.BORDER_CONSTANT))
    
    
    #2. Bilateral,​ ​Gaussian and Median Filtering
    
    #*********Bilateral filtering funtion - mask size 5 × 5 *****************
    
    #own funtion
    image_own = bilateral_fil(img, 200.0, 200.0)
    cv2.imwrite("own_bilateralfilter.png", image_own)
    print("Image from bilateralfilter own function is saved")
    
    #openCV funtion for bilateralfilter
    fil_img_OpenCV = cv2.bilateralFilter(img, 5, 200.0, 200.0, borderType=cv2.BORDER_CONSTANT)
    cv2.imwrite("OpenCV_bilateralFilter_image.png", fil_img_OpenCV)
    print("Image from OpenCV is saved")
    print('\n')
    
    
    #********* Gaussian filter funtion - mask size 5 × 5*****************
    
    #own funtion
    new_img_gauss = Gaussian_fil(img, 1)
    cv2.imwrite("own_Gaussian_filter_image.png", new_img_gauss)
    print("Image from Gaussin own funtion is saved")
    
    
    #openCV funtion
    new_img = cv2.GaussianBlur(img, (5,5),0, borderType=cv2.BORDER_CONSTANT)
    cv2.imwrite("OpenCV_Gaussianfilter_image.png", new_img)
    print("Image from Gaussian openCV funtion is saved")
    print('\n')
        
        
    
    #********* Median filter funtion*****************
    
    #own funtion
    median_img = Median_fil(img, 5)
    cv2.imwrite("own_median_filter_image.png",  median_img)
    print("Image from median own funtion is saved")
    
    
    #openCV funtion
    median_img2 = cv2.medianBlur(img, 5)
    cv2.imwrite("OpenCV_median_filter_image.png", median_img2)
    print("Image from median openCV funtion is saved")
    
if __name__ == "__main__":
    main()

[[ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [20. 30. 30. 30. 30. 30. 30. 20.]]



[[-20. -30. -30. -30. -30. -30. -30. -20.]
 [  0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.]
 [ 20.  30.  30.  30.  30.  30.  30.  20.]]


c:\users\asus\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:39: RuntimeWarning: overflow encountered in ubyte_scalars


Image from bilateralfilter own function is saved
Image from OpenCV is saved


Image from Gaussin own funtion is saved
Image from Gaussian openCV funtion is saved


Image from median own funtion is saved
Image from median openCV funtion is saved
